# Multi-class Classification of Adult Dataset

This notebook contains the multi-class classification of Adult Dataset on target 'marital-status', using 4 different classifiers:
<ol>
    <li> Gradient boosting </li>
    <li> Random Forest </li>
    <li> Linear SVC </li>
    <li> Logistic Regression </li>
</ol>
We focus on recreating the results from the paper "DO NOT DISTURB? Classifier Behavior on Perturbed Datastes" that analyzes the impact of data anonymization to results of the classification tasks. We first analyize here how to preprocess the data and examine the results that we will later compare to results from anonymized datasets.

In [46]:
target = 'education-num'
print(target)

education-num


In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle

from sklearn import metrics, preprocessing, model_selection
from sklearn.ensemble import GradientBoostingClassifier as GradientBoosting, RandomForestClassifier as RandomForest
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [48]:
# read data
if target == 'marital-status':
    filepath = "../data/adult_all.csv"
elif target == 'education-num':
    filepath = "../data/adult_grouped.csv"

dataset = pd.read_csv(filepath, sep=r'\s*,\s*', na_values="*", engine='python', index_col=0)

We can have a look at our dataset by printing first 5 rows.

In [49]:
dataset.head()

age         workclass  education-num      marital-status  \
NodeID                                                             
0        39         State-gov              3       Never-married   
1        50  Self-emp-not-inc              3  Married-civ-spouse   
2        38           Private              2            Divorced   
3        53           Private              2  Married-civ-spouse   
4        28           Private              3  Married-civ-spouse   

               occupation   relationship   race     sex  capital-gain  \
NodeID                                                                  
0            Adm-clerical  Not-in-family  White    Male          2174   
1         Exec-managerial        Husband  White    Male             0   
2       Handlers-cleaners  Not-in-family  White    Male             0   
3       Handlers-cleaners        Husband  Black    Male             0   
4          Prof-specialty           Wife  Black  Female             0   

        capital-loss  hours-per-week native-country income  
NodeID                                                      
0                  0              40  United-States  <=50K  
1                  0              13  United-States  <=50K  
2                  0              40  United-States  <=50K  
3                  0              40  United-States  <=50K  
4                  0              40         Non-US  <=50K

In order to work with classifiers from sklearn, we will need to convert the categorical features into numerical features. We do it with the following function.

In [50]:
# Preprocessing
def number_encode_features(ds):
    result = ds.copy()
    encoders = {}
    for feature in result.columns:
        if result.dtypes[feature] == np.object:
            encoders[feature] = preprocessing.LabelEncoder()
            result[feature] = encoders[feature].fit_transform(result[feature].astype(str))
    return result, encoders

dataset_encoded, encoders = number_encode_features(dataset)

We can see how our encoded dataset looks like now.

In [51]:
dataset_encoded.head()

age  workclass  education-num  marital-status  occupation  \
NodeID                                                              
0        39          5              3               4           0   
1        50          4              3               2           3   
2        38          2              2               0           5   
3        53          2              2               2           5   
4        28          2              3               2           9   

        relationship  race  sex  capital-gain  capital-loss  hours-per-week  \
NodeID                                                                        
0                  1     4    1          2174             0              40   
1                  0     4    1             0             0              13   
2                  1     4    1             0             0              40   
3                  0     2    1             0             0              40   
4                  5     2    0             0             0              40   

        native-country  income  
NodeID                          
0                    1       0  
1                    1       0  
2                    1       0  
3                    1       0  
4                    0       0

As noted before, as a target for classification task we will use the attribute 'marital-status'.

In [52]:
y = dataset_encoded[target]
X = dataset_encoded.drop(target, axis=1)

We define two functions for calculating metrics.

In [53]:
def f1_micro(clf, X, y):
    # cross validation scores on number encoded data
    scores = model_selection.cross_val_score(clf, X, y, cv=10, scoring='f1_micro')
    print("F1 score: %0.2f (+/- %0.2f)" 
      % (scores.mean(), scores.std() * 2))
    return scores.mean()

In [54]:
def f1_macro(clf, X, y):
    # cross validation scores on number encoded data
    scores = model_selection.cross_val_score(clf, X, y, cv=10, scoring='f1_macro')
    print("F1 score: %0.2f (+/- %0.2f)" 
      % (scores.mean(), scores.std() * 2))
    return scores.mean()

In [55]:
scores = {}

## 1. Gradient Boosting

Score from the paper: 0.85

In [56]:
# Gradient Boosting
clf = GradientBoosting(random_state=0)

In [57]:
scores['Gradient Boosting'] = f1_micro(clf, X, y)

F1 score: 0.63 (+/- 0.02)


## 2. Random Forest

Score from the paper: 0.84

In [58]:
clf = RandomForest(random_state=0)
scores['Random Forest'] = f1_micro(clf, X, y)

F1 score: 0.57 (+/- 0.02)


## 3. Linear SVC

Score from the paper: 0.85

In [59]:
# Linear SVC
clf = LinearSVC(random_state=0)

In [60]:
f1_micro(clf, X, y)

F1 score: 0.40 (+/- 0.21)


0.40142370578849473

The result is not satisfactory so we will try a different preprocessing method for classification using Linear SVC. We will use 'one-hot encoding' for categorical features and analyze the results.

In [61]:
# we can try with binary encoded features
# Target will be 'marital-status'
y = dataset[target]
X = dataset.drop(target, axis=1)

In [62]:
X = pd.get_dummies(X)
X.shape

(30162, 49)

Our dataset now has 43 features, all categorical ones turned into binary.

In [63]:
X.columns

Index(['age', 'capital-gain', 'capital-loss', 'hours-per-week',
       'workclass_Federal-gov', 'workclass_Local-gov', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay',
       'marital-status_Divorced', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech-support', 'occupation_Transport-moving',
       'relationship_Husband', 'relatio

In [64]:
del X['sex_Male'], X['income_>50K'], X['native-country_Non-US']

In [65]:
# Linear SVC
clf = LinearSVC(random_state=0)

In [66]:
scores['Linear SVC'] = f1_micro(clf, X, y)

F1 score: 0.43 (+/- 0.15)


## 4. Logistic Regression

Score from the paper: 0.81

For Logistic Regression we will use one-hot encoded features as well.

In [67]:
clf = LogisticRegression(random_state=0)

In [68]:
scores['Logistic Regression'] = f1_micro(clf, X, y)

F1 score: 0.61 (+/- 0.02)


## Conclusion

Even though we don't have information of tools, preprocessing methods and parameters used to obtain the classification results from paper, our results are close enough to be used for further analysis.

### Saving the results

In [69]:
filename = '../data/output/ARX/' + target + '/classification-res/adult_multiclass_full'
outfile = open(filename,'wb')

In [70]:
pickle.dump(scores, outfile)
outfile.close()